### Imports

In [1]:
import pandas as pd
#from sql import engine #adjust this as necessary to match your sql.py connection methods
import psycopg2

### Upload all files

In [2]:
# Load all necessary data
df = pd.read_excel("/Users/pascalsommer/neue fische stuff/capstone_final/data/Cars.xlsx")
df2 = pd.read_csv("/Users/pascalsommer/neue fische stuff/capstone_final/places_final_new.csv")
abg = pd.read_csv('/Users/pascalsommer/neue fische stuff/capstone_final/data/Arbeitslosenquote - Arbeitslosenquote.csv')

### City cars DataFrame

In [3]:
#Rename the column
df.rename(columns = {"Statistische Kennziffer und Zulassungsbezirk":"registration district","insgesamt":"passenger car"}, inplace=True)

In [4]:
# Change the column to string
df["registration district"] = df["registration district"].astype("string")


# Remove the whitspaces
df["registration district"] = df["registration district"].str.strip()

# Drop all null values
df.dropna(subset=["registration district"], inplace=True)

In [5]:
# Use the lambda function to split the cities how we need them
df["registration district"] = df["registration district"].apply(lambda x: str(x).split("  ")[1].split(",")[0])

# Use the string function title() to capitalize all sentence at the beginning of a word
df["registration district"] = df["registration district"].apply(lambda x: x.lower())

# Change the words ue and oe to ü and ö (Umlaute)
# df["registration district"] = df["registration district"].str.replace('ue','ü').str.replace('oe','ö')

In [6]:
# List of our Cities
city_cars_list = ['hamburg','berlin','koeln','muenchen','frankfurt am main']

# Create a new DataFrame where only our cities with the important values are in
city_cars = df[df["registration district"].isin(city_cars_list)]

# Drop the Munich - Randbezirke row with the index 60
city_cars.drop(index=60,axis=1,inplace=True)

# Reset the index
city_cars = city_cars.reset_index()

# Drop the old index column
city_cars.drop('index',axis=1,inplace=True)

/opt/homebrew/Caskroom/miniconda/base/envs/eda_env/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [7]:
# Population density by age
city_cars["age 18 - 24"] = [119731,264084,146312,58178,92243]
city_cars["age 25 - 34"] = [275743,613696,301802,137963,186045]
city_cars["age 35 - 44"] = [221506,530715,266363,120998,154761]
city_cars["age 45 - 54"] = [205658,482906,261156,110428,156601] 
city_cars["age 55 - 64"] = [166733,466691,224375,87402,133456]
city_cars["age 65 - 74"] = [119222,325994,152595,58926,89354]
city_cars["age above 75"] = [139863,379531,183764,60097,100067]

age_range = ['age 18 - 24','age 25 - 34','age 35 - 44','age 45 - 54','age 55 - 64','age 65 - 74','age above 75']

city_cars["total population above 18"] = city_cars[age_range].sum(axis=1)

In [8]:
# calculate the cars per cap column
city_cars["cars per capita"] = city_cars["passenger car"]/city_cars["total population above 18"]

In [9]:
# Income per capita
city_cars["income per capita"] = [32.766,20.972,25.029,23.436,22.402]

In [10]:
# lat and lon for join in Sql
city_cars['latitude'] = [48.137154,52.518611,53.550556,50.116667,50.936389]
city_cars['longitude'] = [11.576124,13.408333,9.993333,8.683333,6.952778]

In [11]:
# column order
city_cars = city_cars[["registration district","passenger car","total population above 18","age 18 - 24","age 25 - 34","age 35 - 44","age 45 - 54","age 55 - 64","age 65 - 74","age above 75","cars per capita","income per capita","latitude","longitude"]]

In [12]:
# Print to test 
# 0,6 autos pro person könnte aber verzerrt sein durch Autovermietung
city_cars

,registration district,passenger car,total population above 18,age 18 - 24,age 25 - 34,age 35 - 44,age 45 - 54,age 55 - 64,age 65 - 74,age above 75,cars per capita,income per capita,latitude,longitude
0,muenchen,732045,1248456,119731,275743,221506,205658,166733,119222,139863,0.586360,32.766,48.137154,11.576124
1,berlin,1234645,3063617,264084,613696,530715,482906,466691,325994,379531,0.403002,20.972,52.518611,13.408333
2,hamburg,805780,1536367,146312,301802,266363,261156,224375,152595,183764,0.524471,25.029,53.550556,9.993333
3,frankfurt am main,342431,633992,58178,137963,120998,110428,87402,58926,60097,0.540119,23.436,50.116667,8.683333
4,koeln,492402,912527,92243,186045,154761,156601,133456,89354,100067,0.539603,22.402,50.936389,6.952778


In [13]:
# city_cars["income per capita"] = city_cars["income per capita"].apply(lambda x: str(x) + " €")

### City size DataFrame

In [14]:
df_size_F = df2.query("City == 'frankfurt am main'")
df_size_HH = df2.query("City == 'hamburg'")
df_size_M = df2.query("City == 'muenchen'")
df_size_K = df2.query("City == 'koeln'")
df_size_B = df2.query("City == 'berlin'")
    

In [15]:
df2

,Unnamed: 0,index,Typ,Geometry_Type,Coordinates,Postcode,City,Population,Size,Population_Density,Country
0,0,0,Feature,Polygon,"[[[11.394678, 48.147287], [11.394845, 48.14654...",81248,muenchen,121,1.984763,60,Deutschland
1,1,1,Feature,Polygon,"[[[8.672537, 50.11264], [8.673634, 50.111517],...",60315,frankfurt am main,0,0.017481,0,Deutschland
2,2,8,Feature,Polygon,"[[[8.672617, 50.11164], [8.672931, 50.111289],...",60312,frankfurt am main,0,0.001829,0,Deutschland
3,3,24,Feature,Polygon,"[[[8.66955, 50.115843], [8.670234, 50.11568], ...",60306,frankfurt am main,0,0.005756,0,Deutschland
4,4,1118,Feature,Polygon,"[[[8.672014, 50.110757], [8.672315, 50.110397]...",60310,frankfurt am main,0,0.007068,0,Deutschland
...,...,...,...,...,...,...,...,...,...,...,...
447,447,7961,Feature,Polygon,"[[[9.970048, 53.567306], [9.974557, 53.565203]...",20146,hamburg,10420,0.931521,11186,Deutschland
448,448,7962,Feature,Polygon,"[[[9.986446, 53.571122], [9.986498, 53.571001]...",20148,hamburg,5236,1.166056,4490,Deutschland
449,449,8045,Feature,Polygon,"[[[13.36126, 52.398468], [13.361407, 52.397665...",12307,berlin,11924,4.168932,2860,Deutschland
450,450,8048,Feature,Polygon,"[[[9.798504, 53.475127], [9.799123, 53.475124]...",21147,hamburg,19824,15.501663,1278,Deutschland


In [16]:
df_size_F = df_size_F.groupby(["City"], as_index=False)["Size"].sum()
df_size_HH = df_size_HH.groupby(["City"], as_index=False)["Size"].sum()
df_size_M = df_size_M.groupby(["City"], as_index=False)["Size"].sum()
df_size_K = df_size_K.groupby(["City"], as_index=False)["Size"].sum()
df_size_B = df_size_B.groupby(["City"], as_index=False)["Size"].sum()


In [17]:
df_size_F

,City,Size
0,frankfurt am main,248.407973


In [18]:
city_size = pd.concat([df_size_M,df_size_B,df_size_HH,df_size_F,df_size_K])

In [19]:
city_size = city_size.rename(columns={"City":"registration district","Size":"size"})

In [20]:
city_size.reset_index(drop=True, inplace=True)

In [21]:
city_size['registration district'] = city_size['registration district'].replace(['Frankfurt'],'Frankfurt Am Main')

In [22]:
city_size

,registration district,size
0,muenchen,310.904727
1,berlin,889.412893
2,hamburg,794.624370
3,frankfurt am main,248.407973
4,koeln,404.856749


### Arbeitslosenquote DataFramen

In [23]:
abg.rename(columns = {'Stadt':'registration district', 'Arbeitslosenquote':'unemployment rate in percent'}, inplace = True)

In [24]:
abg['registration district'] = abg['registration district'].replace(['Frankfurt'],'frankfurt am main')
abg['registration district'] = abg['registration district'].apply(lambda x: x.lower().replace('ö','oe').replace('ü','ue'))

In [25]:
abg

,registration district,unemployment rate in percent
0,hamburg,7.6
1,berlin,9.7
2,koeln,9.2
3,frankfurt am main,6.7
4,muenchen,4.8


### Merge of all tables

In [26]:
city_cars = city_cars.merge(city_size, how='left' , on='registration district').merge(abg, how='left' , on='registration district')

city_cars

,registration district,passenger car,total population above 18,age 18 - 24,age 25 - 34,age 35 - 44,age 45 - 54,age 55 - 64,age 65 - 74,age above 75,cars per capita,income per capita,latitude,longitude,size,unemployment rate in percent
0,muenchen,732045,1248456,119731,275743,221506,205658,166733,119222,139863,0.586360,32.766,48.137154,11.576124,310.904727,4.8
1,berlin,1234645,3063617,264084,613696,530715,482906,466691,325994,379531,0.403002,20.972,52.518611,13.408333,889.412893,9.7
2,hamburg,805780,1536367,146312,301802,266363,261156,224375,152595,183764,0.524471,25.029,53.550556,9.993333,794.624370,7.6
3,frankfurt am main,342431,633992,58178,137963,120998,110428,87402,58926,60097,0.540119,23.436,50.116667,8.683333,248.407973,6.7
4,koeln,492402,912527,92243,186045,154761,156601,133456,89354,100067,0.539603,22.402,50.936389,6.952778,404.856749,9.2


In [27]:
city_cars['population density'] = round(city_cars['total population above 18']/city_cars["size"])

In [28]:
city_cars['population density'] = city_cars['population density'].astype('int')

In [29]:
points=[]
for i in city_cars['population density']:
    if i >= 0 and i <=(city_cars['population density'].max()*0.1):
        points.append(1)
    elif i >(city_cars['population density'].max()*0.1) and i <=(city_cars['population density'].max()*0.2):
        points.append(2)
    elif i >(city_cars['population density'].max()*0.2) and i <=(city_cars['population density'].max()*0.3):
        points.append(3)
    elif i >(city_cars['population density'].max()*0.3) and i <=(city_cars['population density'].max()*0.4):
        points.append(4)
    elif i >(city_cars['population density'].max()*0.4) and i <=(city_cars['population density'].max()*0.5):
        points.append(5)
    elif i >(city_cars['population density'].max()*0.5) and i <=(city_cars['population density'].max()*0.6):
        points.append(6)
    elif i >(city_cars['population density'].max()*0.6) and i <=(city_cars['population density'].max()*0.7):
        points.append(7)
    elif i >(city_cars['population density'].max()*0.7) and i <=(city_cars['population density'].max()*0.8):
        points.append(8)
    elif i >(city_cars['population density'].max()*0.8) and i <=(city_cars['population density'].max()*0.9):
        points.append(9)
    elif i >(city_cars['population density'].max()*0.9) and i <=(city_cars['population density'].max()):
        points.append(10)
city_cars['pop_dens_points']=points

In [30]:
# dataframe interprets thousand splitter as a comma so function interprets the number as a float not as a thousand (32.766 gets interpreted as 32 COMMA 766)
points=[]
for i in city_cars['income per capita']:
    if i >= 27.503: #top10
        points.append(10)
    elif i < 27.503 and i >= 25.338: #11-50
        points.append(9)
    elif i < 25.338 and i >= 24.272: #51-100
        points.append(8)
    elif i < 24.272 and i >= 23.290: #101-150
        points.append(7)
    elif i < 23.290 and i >= 22.561: #151-200
        points.append(6)
    elif i < 22.561 and i >= 21.859: #201-250
        points.append(5)
    elif i < 21.859 and i >= 20.716: #251-300
        points.append(4)
    elif i < 20.716 and i >= 19.740: #301-350
        points.append(3)
    elif i < 19740 and i >= 17.126:
        points.append(2)
    elif i < 17.126:
        points.append(1)
city_cars['income_p_c_points']=points

In [31]:
points=[]
min = 3
max= 15
for i in city_cars['unemployment rate in percent']:
    if i >= min and i <= min+((max-min)*0.1):
        points.append(10)
    elif i > min+((max-min)*0.1) and i <= min+((max-min)*0.2):
        points.append(9)
    elif i > min+((max-min)*0.2) and i <= min+((max-min)*0.3):
        points.append(8)
    elif i > min+((max-min)*0.3) and i <= min+((max-min)*0.4):
        points.append(7)
    elif i > min+((max-min)*0.4) and i <= min+((max-min)*0.5):
        points.append(6)
    elif i > min+((max-min)*0.5) and i <= min+((max-min)*0.6):
        points.append(5)
    elif i > min+((max-min)*0.6) and i <= min+((max-min)*0.7):
        points.append(4)
    elif i > min+((max-min)*0.7) and i <= min+((max-min)*0.8):
        points.append(3)
    elif i > min+((max-min)*0.8) and i <= min+((max-min)*0.9):
        points.append(2)
    elif i > min+((max-min)*0.9) and i <= min+((max-min)):
        points.append(1)
city_cars['unempl_rate_points']=points

In [46]:
points=[]
min = 0.35
max= 0.7
for i in city_cars['cars per capita']:
    if i >= min and i <= min+((max-min)*0.1):
        points.append(1)
    elif i > min+((max-min)*0.1) and i <= min+((max-min)*0.2):
        points.append(2)
    elif i > min+((max-min)*0.2) and i <= min+((max-min)*0.3):
        points.append(3)
    elif i > min+((max-min)*0.3) and i <= min+((max-min)*0.4):
        points.append(4)
    elif i > min+((max-min)*0.4) and i <= min+((max-min)*0.5):
        points.append(5)
    elif i > min+((max-min)*0.5) and i <= min+((max-min)*0.6):
        points.append(6)
    elif i > min+((max-min)*0.6) and i <= min+((max-min)*0.7):
        points.append(7)
    elif i > min+((max-min)*0.7) and i <= min+((max-min)*0.8):
        points.append(8)
    elif i > min+((max-min)*0.8) and i <= min+((max-min)*0.9):
        points.append(9)
    elif i > min+((max-min)*0.9) and i <= min+((max-min)):
        points.append(10)
city_cars['cars_p_c_points']=points

In [47]:
city_cars

,registration district,passenger car,total population above 18,age 18 - 24,age 25 - 34,age 35 - 44,age 45 - 54,age 55 - 64,age 65 - 74,age above 75,...,income per capita,latitude,longitude,size,unemployment rate in percent,population density,pop_dens_points,income_p_c_points,unempl_rate_points,cars_p_c_points
0,muenchen,732045,1248456,119731,275743,221506,205658,166733,119222,139863,...,32.766,48.137154,11.576124,310.904727,4.8,4016,10,10,9,7
1,berlin,1234645,3063617,264084,613696,530715,482906,466691,325994,379531,...,20.972,52.518611,13.408333,889.412893,9.7,3445,9,4,5,2
2,hamburg,805780,1536367,146312,301802,266363,261156,224375,152595,183764,...,25.029,53.550556,9.993333,794.624370,7.6,1933,5,8,7,5
3,frankfurt am main,342431,633992,58178,137963,120998,110428,87402,58926,60097,...,23.436,50.116667,8.683333,248.407973,6.7,2552,7,7,7,6
4,koeln,492402,912527,92243,186045,154761,156601,133456,89354,100067,...,22.402,50.936389,6.952778,404.856749,9.2,2254,6,5,5,6


### Creating upload in sql and a csv file

In [35]:
# Export to csv
city_cars.to_csv('/Users/pascalsommer/neue fische stuff/capstone_final/city_cars_with_points.csv', index=False)

In [36]:
# # # Specify which table within your database you want to push your data to. Give your table an unambiguous name.
# table_name = 'city_cars'
# schema = 'muc_analytics_21_1'
# # If the specified table doesn't exist yet, it will be created
# # With 'replace', your data will be replaced if the table already exists.
# # This will take a minute or two...

# # Write records stored in a dataframe to SQL database
# if engine!=None:
#     try:
#         city_cars.to_sql(name=table_name, # Name of SQL table
#                         con=engine, # Engine or connection
#                         if_exists='replace', # Drop the table before inserting new values 
#                         schema=schema, # Use schmea that was defined earlier
#                         index=False, # Write DataFrame index as a column
#                         chunksize=5000, # Specify the number of rows in each batch to be written at a time
#                         method='multi') # Pass multiple values in a single INSERT clause
#         print(f"The {table_name} table was imported successfully.")
#     # Error handling
#     except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
#         engine = None